In [ ]:
"""
This file consists of implementation of “Robust k-means++” paper. 
Author: Praneeth Kacham
"""

In [1]:
import numpy as np
import collections
import statistics
from sklearn.cluster import KMeans

In [2]:
def sq_distance(x,y):
    diff = x-y
    return np.inner(diff,diff)

In [3]:
def farthest(distances, num_outliers):
    """Takes the distances array and returns indices of the farthest 'num_outliers' and indices of remaining points using these distances"""
    idx = np.argpartition(distances,-num_outliers)
    return idx[-num_outliers:],idx[:-num_outliers]

In [4]:
def kmeanspp(points,k):
    """
    points - nxd array of points
    k - number of clusters
    """
    m,_ = points.shape
    center_indices = [np.random.choice(m)]
    c_point = points[center_indices[0]]
    sq_distances = np.zeros(m)
    for i in range(m):
        sq_distances[i] = sq_distance(points[i],c_point)
    for i in range(k-1):
        sum_ = np.sum(sq_distances)
        prob = sq_distances/sum_
        c_index = np.random.choice(np.arange(m),1,p=prob)
        center_indices.append(c_index[0])
        c_point = points[c_index]
        for j in range(m):
            c_dist = sq_distance(points[j],c_point)
            if( c_dist < sq_distances[j]):
                sq_distances[j] = np.float(c_dist)
    return center_indices, sq_distances

In [5]:
def random_initialization(points,k):
    """
    points - nxd array
    k : int - number of clusters
    """
    center_indices = np.random.choice(np.arange(len(points)),k,replace=False)
    sq_distances = [min([sq_distance(point, points[center]) for center in center_indices]) for point in points]
    return center_indices, sq_distances

In [6]:
def cost(points,center_indices):
    """
    points - nxd array
    center_indices : [int] 
    """
    m,_ = points.shape
    cost = sum([min([sq_distance(points[i],points[center_index]) for center_index in center_indices]) for i in range(m)])
    return cost

In [7]:
# Returns the probability of picking each point
def mixture_sampling(points,center_indices,alpha,distances_squared):
    """
    points : mxd array of points
    center_indices : [int]
    alpha : float
    distances_squared : [float] of size m
    """
# alpha is the coefficient of Uniform Sampling. If alpha = 0, then k-means++ if alpha = 1, then uniform
    m = len(points)
    total_distances_squared = sum(distances_squared)
    probabilities = [(1-alpha)*x/total_distances_squared + alpha/m for x in distances_squared]
    return probabilities

In [8]:
def kmeans_with_mixture(points,k,delta,alpha):
    """
    points : mxd array 
    k : int - number of clusters
    delta : float
    alpha : float
    
    Returns : 
    center_indices : 1 + (k-1)/delta size array of ints
    cluster_number : [int] of size m : cluster_number[i] denotes the index of the nearest point in center_indices
    distances : [float] of size m : distance to nearest cluster center
    """
    m,_ = points.shape
    distances = np.zeros(m)
    center_indices = [np.random.choice(m)] #np.random.c #Choosing first centre randomly
    cluster_number = [center_indices[0] for i in range(m)]
    for i in range(m):
        distances[i] = sq_distance(points[i],points[center_indices[0]])
    for i in range(k-1): # Choosing 1/delta centres from mixture distribution in each iteration
        probabilities = mixture_sampling(points,center_indices,alpha,distances)
        a = np.random.choice(np.arange(m),int(1/delta),p=probabilities)
        a = list(set(a))
        center_indices.extend(a)
        for j in range(m):
            for new in a:
                dist = sq_distance(points[j],points[new])
                if (dist < distances[j]):
                    cluster_number[j] = new
                    distances[j] = dist
    return center_indices, cluster_number, distances

In [9]:
def weights_of_clusters(points, center_indices, cluster_number):
    """
    points : mxd array
    center_indices : [int] - list of indices of centres
    cluster_number : [int] - index of nearest centre to each point
    
    Returns:
    weights : dictionary : center_indices -> int : Take care that each value is non-zero
    """
    weights = {}
    for index in center_indices:
        weights[index] = 0
    for i in range(len(cluster_number)):
        weights[cluster_number[i]] += 1
    for index in center_indices:
        if weights[index] == 0:
            print("Error : The following index is not closest to any")
            print(index)
    return weights

In [10]:
def weighted_probabilities(points, indices, centers, weights):
    """
    Returns the probabilities for weighted D^2 sampling given the current centers
    """
    costs = []
    for i in range(len(indices)):
        current_point = points[indices[i]]
        current_weight = weights[indices[i]] 
        min_weighted_distance = float("inf")
        for j in centers:
            center_point = points[j]
            weighted_distance_square = current_weight*np.dot(center_point-current_point,center_point-current_point)
            if weighted_distance_square < min_weighted_distance:
                min_weighted_distance = weighted_distance_square
        costs.append(min_weighted_distance)
    total_dis_squared = sum(costs)
    probs = [cost/total_dis_squared for cost in costs]
    return probs

In [11]:
def weighted_kmeans(points,indices,weights,k,repetitions=100):
    """
    Given the points, indices which should be considered for clustering, weights of each points, k, performs many repetitions
    of weighted_kmeans and returns the best weighted clustering
    """
    current_cost = np.float("inf")
    best_centers = []
    for rep in range(repetitions):
        centers = (np.random.choice(indices,1))
        centers = centers.tolist()
        for i in range(k-1):
            probabilities = weighted_probabilities(points,indices,centers,weights)
            centers.extend((np.random.choice(indices,1,p=probabilities)).tolist())
        cost = 0
        for i in range(len(indices)):
            nearest_distance = float("inf")
            for j in centers:
                if sq_distance(points[j],points[indices[i]]) < nearest_distance:
                    nearest_distance = sq_distance(points[indices[i]],points[j])
            cost += nearest_distance*weights[indices[i]]
        if(cost < current_cost):
            best_centers = centers
            current_cost = cost
    return best_centers

In [12]:
def generate_data_gaussian(k, num_points, num_outliers, dimension, inlier_box, outlier_box):
    """
    Generates k cluster_centres, each with num_points/k points chosen with gaussian distribution of unit variance around the cluster centre.
    Additionally adds num_outliers number of random points within the outlier_box. Then finds the true_outliers as the farthest num_outliers no. of points
    from cluster_centres
    """
    num_repeats = int(num_points/k)
    points = []
    cov = np.ones(dimension)
    cov = np.diag(cov)
    centers = []
    cost = 0.0
    for i in range(k):
        current = np.random.uniform(0,inlier_box,dimension)
        centers.append(current)
        new_points = np.random.multivariate_normal(current,cov,num_repeats).tolist()
        for point in new_points:
            cost += sq_distance(point,current)
        points.extend(new_points)
    lb = -1.0*(outlier_box - inlier_box)/2.0
    ub = inlier_box + (outlier_box - inlier_box)/2.0
    for i in range(num_outliers):
        current = np.random.uniform(lb,ub,dimension)
        points.append(current)
    points = np.array(points)
    m,_ = points.shape
    distances = [] # Denotes the distance to the nearest cluster center
    cluster = [] # Denotes the cluster to which the point belongs
    for i in range(m):
        distance = min([(sq_distance(points[i],centers[j]),j) for j in range(len(centers))])
        distances.append((distance[0],i))
        cluster.append(distance[1])
    distances.sort()
    true_outliers = [a[1] for a in distances[-1*num_outliers:]]
    for j in true_outliers:
        cluster[j] = -1
    return points, true_outliers, cluster

In [13]:
def weights(cluster_indices,inliers): #Cluster center of cluster to which a point belongs to
    """
    Finds the weights of each of the points i.e., no. of points for which this is the closest point in chosen centres
    """
    w = {}
    for i in inliers:
        if cluster_indices[i] not in w:
            w[cluster_indices[i]] = 0
        w[cluster_indices[i]] += 1
    return w

In [14]:
def robust_cost(points,center_indices,num_outliers):
    """
    Finds the cost of the kmeans solution excluding farthest num_outliers no. of points
    """
    m,_ = points.shape
    distances = []
    for i in range(m):
        distances.append((min([sq_distance(points[i],points[j]) for j in center_indices]),i))
    distances.sort()
    farthest = distances[-1*num_outliers:]
    return (sum([dist[0] for dist in farthest]),[dist[1] for dist in farthest])

In [15]:
def distance_from_centres(points, centre_indices):
    distances = [min([sq_distance(points[i],points[centre_index]) for centre_index in centre_indices]) for i in range(len(points))]
    return distances

In [16]:
def robust_kmeanspp(data, k, num_outliers, alpha, delta, true_outliers, repetitions):
    outliers_recovereds = []
    costs = []
    for rep in range(repetitions):
        # First we select k/delta points using kmeans_with_mixture
        initial_centres, clusters, distances = kmeans_with_mixture(data, k, delta, alpha)
        # Now find farthest "num_outliers" 
        initial_outliers, initial_inliers = farthest(distances, num_outliers)
        # Now find weights of clusters
        weights_of_clusters = weights(clusters, initial_inliers)
        # The following heuristic can be used to remove the centres which have low weight.
#         to_remove = []
#         for i in initial_centres:
#             if weights_of_clusters[i] == 1:
#                 to_remove.append(i)
#         initial_centres = list(set(initial_centres)-set(to_remove))
        weighted_kmeans_results = weighted_kmeans(data, initial_centres, weights_of_clusters,k,60)
        # Now, we have k centres. Remove farthest "num_outliers" from these k centres
        distances_from_final_centres = distance_from_centres(data, weighted_kmeans_results)
        outliers,inliers = farthest(distances_from_final_centres,num_outliers)
        kmeans = KMeans(n_clusters=k,init=data[weighted_kmeans_results],n_init=1).fit(data[inliers])
        cost = kmeans.inertia_
        predicted_outliers = outliers
        outliers_recovered = len(set(predicted_outliers) & set(true_outliers))
        outliers_recovereds.append(outliers_recovered)
        costs.append(cost)
    average_outliers_recovered = statistics.mean(outliers_recovereds)
    max_outliers_recovered = max(outliers_recovereds)
    median_outliers_recovered = statistics.median(outliers_recovereds)
    average_cost = statistics.mean(costs)
    median_cost = statistics.median(costs)
    min_cost = min(costs)
    return (max_outliers_recovered, average_outliers_recovered, median_outliers_recovered, min_cost, average_cost, median_cost)

In [17]:
def kmeanspp_outliers(data, k, num_outliers, true_outliers, repetitions):
    m,_ = data.shape
    outliers_recovereds = []
    costs = []
    for rep in range(repetitions):
        centres = [np.random.choice(m)]
        distances = np.array([sq_distance(data[i],data[centres[0]]) for i in range(m)])
        clusters = [centres[0] for i in range(m)]
        for i in range(k-1):
            distance_sum = np.sum(distances)
            probabilities = [distances[i]/distance_sum for i in range(m)]
            c_index = np.random.choice(np.arange(m),1,p=probabilities)
            centres.append(c_index[0])
            for j in range(m):
                distance_from_new_centre = sq_distance(data[c_index,:], data[j,:])
                if distance_from_new_centre < distances[j]:
                    distances[j] = distance_from_new_centre
                    clusters[j] = c_index
        outliers, inliers = farthest(distances, num_outliers)
        # Now run lloyd's on the remaining points
        kmeans = KMeans(n_clusters=k,init=data[centres],n_init=1).fit(data[inliers])
        cost = kmeans.inertia_
        outliers_recovered = len(set(true_outliers)&set(outliers))
        outliers_recovereds.append(outliers_recovered)
        costs.append(cost)
    average_outliers_recovered = statistics.mean(outliers_recovereds)
    max_outliers_recovered = max(outliers_recovereds)
    median_outliers_recovered = statistics.median(outliers_recovereds)
    average_cost = statistics.mean(costs)
    median_cost = statistics.median(costs)
    min_cost = min(costs)
    return (max_outliers_recovered, average_outliers_recovered, median_outliers_recovered, min_cost, average_cost, median_cost)

In [18]:
def random_outliers(data, k, num_outliers, true_outliers, repetitions):
    m,_ = data.shape
    outliers_recovereds = []
    costs = []
    for rep in range(repetitions):
        centres = np.random.choice(np.arange(m),k,replace=False)
        distances = [min([sq_distance(data[i],data[centre]) for centre in centres]) for i in range(m)]
        outliers,inliers = farthest(distances, num_outliers)
        kmeans = KMeans(n_clusters=k, init=data[centres], n_init=1).fit(data[inliers])
        cost = kmeans.inertia_
        outliers_recovered = len(set(true_outliers)&set(outliers))
        costs.append(cost)
        outliers_recovereds.append(outliers_recovered)
    average_outliers_recovered = statistics.mean(outliers_recovereds)
    max_outliers_recovered = max(outliers_recovereds)
    median_outliers_recovered = statistics.median(outliers_recovereds)
    average_cost = statistics.mean(costs)
    median_cost = statistics.median(costs)
    min_cost = min(costs)
    return (max_outliers_recovered, average_outliers_recovered, median_outliers_recovered, min_cost, average_cost, median_cost)

## Running Experiments --  Real world Dataset

In [ ]:
def read_data(fileName):
    with open(fileName) as file:
        readCSV = csv.reader(file,delimiter=' ')
        examples = []
        labels = []
        for row in readCSV:
            current = [float(x) for x in row[:-1]]
            labels.append(int(row[-1]))
            examples.append(current)
        examples = np.array(examples)
        labels = np.array(labels)
        return(examples,labels)

In [ ]:
examples,labels = read_data(filename) ## Inout the file name of the dataset

In [ ]:
results = {}
for (k,outliers) in [(5,17),(5,21),(10,17),(10,34),(15,17),(15,51)]:
    result = kmeanspp_outliers(examples, k, outliers, true_outliers, 10)
    results[(k,outliers)] = result
    print((k,outliers))
    print((result))

## Synthetic Data

### An example experiment

In [20]:
n = 1000
d = 2
k = 20
inbox = 100
outbox = 100
outliers = 100
data, true_outliers, cluster = generate_data_gaussian(k, n, outliers, d, inbox, outbox)
results = kmeanspp_outliers(data, k, outliers, true_outliers, 2)
print("k-means++ = "+str(results))
results = random_outliers(data, k, outliers, true_outliers, 2)
print("random = "+str(results))
for delta in [0.05, 0.1]:
    for alpha in [0, 0.25, 0.5, 0.75, 1]:
        results = robust_kmeanspp(data, k, outliers, alpha, delta, true_outliers, 2)
        print("delta = "+str(delta)+" alpha = "+str(alpha)+" "+str(results))

k-means++ = (32, 30, 30.0, 11686.977635238474, 12647.745659383, 12647.745659383)
random = (31, 23, 23.0, 12109.216490527706, 25005.299295364595, 25005.299295364595)
delta = 0.05 alpha = 0 (63, 60, 60.0, 4168.902434899326, 5475.688855591797, 5475.688855591797)
delta = 0.05 alpha = 0.25 (51, 45, 45.0, 6069.304794657046, 7542.132125319416, 7542.132125319416)
delta = 0.05 alpha = 0.5 (60, 55, 55.0, 4366.655562256535, 5527.807565686766, 5527.807565686766)
delta = 0.05 alpha = 0.75 (72, 61, 61.0, 2789.463430399758, 4758.591013713041, 4758.591013713041)
delta = 0.05 alpha = 1 (71, 66.5, 66.5, 2861.4149113447315, 2998.5306503623915, 2998.5306503623915)
delta = 0.1 alpha = 0 (62, 58, 58.0, 4466.805352190808, 4482.477553904404, 4482.477553904404)
delta = 0.1 alpha = 0.25 (50, 46, 46.0, 4593.837460237914, 6033.249642184691, 6033.249642184691)
delta = 0.1 alpha = 0.5 (61, 52.5, 52.5, 4144.608781037582, 5674.432399121444, 5674.432399121444)
delta = 0.1 alpha = 0.75 (62, 60, 60.0, 4241.435289346939,